In [1]:
# !pip install llama-index -U --no-cache-dir --force-reinstall
# !pip install -U llama-index-core
# !pip install -U llama-index-embeddings-azure-openai
# !pip install -U llama-index-llms-azure-openai
# !pip install -U qdrant_client fastembed
# !pip install llama-index-vector-stores-qdrant llama-index-readers-file llama-index-embeddings-fastembed llama-index-llms-openai

In [2]:
import requests
import pandas as pd
import os, yaml, qdrant_client
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.embeddings.voyageai import VoyageEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core import set_global_service_context, ServiceContext, \
                             PromptHelper, StorageContext, VectorStoreIndex, load_index_from_storage

c:\Users\Sanduni\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("secrets.yaml", "r") as file:
    secrets = yaml.safe_load(file)

os.environ["OPENAI_API_KEY"] = secrets.get('OPENAI_API_KEY')
os.environ["VOYAGE_AI_API_KEY"] = secrets.get('VOYAGE_AI_API_KEY')

In [4]:
llm = OpenAI(
            api_key=os.environ["OPENAI_API_KEY"],
            model="gpt-3.5-turbo",
            )

embedding_llm = VoyageEmbedding(
                                model_name="voyage-2", 
                                voyage_api_key=os.environ["VOYAGE_AI_API_KEY"]
                                )

text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
prompt_helper = PromptHelper(
                            context_window=4096,
                            num_output=256,
                            chunk_overlap_ratio=0.1,
                            chunk_size_limit=None,
                            )   

service_context = ServiceContext.from_defaults(
                                            llm=llm,
                                            embed_model=embedding_llm,
                                            text_splitter=text_splitter,
                                            prompt_helper=prompt_helper,
                                            )   

set_global_service_context(service_context)

C:\Users\Sanduni\AppData\Local\Temp\ipykernel_16672\1377437198.py:19: DeprecationWarning: Call to deprecated function (or staticmethod) from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [5]:
# if not os.path.exists("./insurance_index"):
#     documents = SimpleDirectoryReader("data").load_data()

#     client = qdrant_client.QdrantClient(path="./insurance_index")
#     vector_store = QdrantVectorStore(
#                                     client=client,
#                                     collection_name="insurance"
#                                     )
#     storage_context = StorageContext.from_defaults(vector_store=vector_store)
#     index = VectorStoreIndex.from_documents(
#                                             documents,
#                                             storage_context=storage_context
#                                             )

#     # save the index
#     index.storage_context.persist(persist_dir="./insurance_index")

# client = qdrant_client.QdrantClient(path="./insurance_index")
# vector_store = QdrantVectorStore(
#                                 client=client,
#                                 collection_name="insurance"
#                                 )
# storage_context = StorageContext.from_defaults(
#                                                 vector_store=vector_store, 
#                                                 persist_dir="./insurance_index"
#                                                 )
# index = load_index_from_storage(storage_context)
# rerank = SentenceTransformerRerank(
#                                     model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
#                                     top_n=3
#                                     )

# query_engine = index.as_query_engine(
#                                     similarity_top_k=10, 
#                                     node_postprocessors=[rerank] 
#                                     )

if not os.path.exists("./insurance_index"):
    documents = SimpleDirectoryReader("data").load_data()

    client = qdrant_client.QdrantClient(path="./insurance_index")
    vector_store = QdrantVectorStore(
                                    client=client,
                                    collection_name="insurance"
                                    )
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
                                            documents,
                                            storage_context=storage_context
                                            )

    # save the index
    index.storage_context.persist(persist_dir="./insurance_index")

client = qdrant_client.QdrantClient(path="./insurance_index")
vector_store = QdrantVectorStore(
                                client=client,
                                collection_name="insurance"
                                )
storage_context = StorageContext.from_defaults(
                                                vector_store=vector_store, 
                                                persist_dir="./insurance_index"
                                                )
index = load_index_from_storage(storage_context)
rerank = SentenceTransformerRerank(
                                    model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
                                    top_n=3
                                    )

query_engine = index.as_query_engine(
                                    similarity_top_k=10, 
                                    node_postprocessors=[rerank] 
                                    )

In [6]:
def generate_response_df():
    df_q = pd.read_csv("docs/Question for Chatbot.csv")
    df_q['Responses'] = df_q['Questions'].apply(lambda x: str(query_engine.query(x)))
    df_q.to_csv("docs/Response from Chatbot.csv", index=False)

In [7]:
query_engine.query("What is the best insurance for a new car?")

Response(response='Comprehensive insurance would be the best insurance for a new car.', source_nodes=[NodeWithScore(node=TextNode(id_='a4a6588c-57cb-487a-8aeb-120858999278', embedding=None, metadata={'page_label': '13', 'file_name': 'motor-english-policy-book-2023.pdf', 'file_path': '/Users/isurusudarsri/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/Assignments/insuarance-chatbot/data/motor-english-policy-book-2023.pdf', 'file_type': 'application/pdf', 'file_size': 350502, 'creation_date': '2024-05-06', 'last_modified_date': '2024-05-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5ee203b1-701b-4939-86da-3729df4115e7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '13', 'file_nam

## Inference

In [8]:
user_url = "http://127.0.0.1:8000/chat/fetch-api"
response_url = "http://127.0.0.1:8000/chat/reply?uuid={}&message={}"

In [9]:
while True:
    user_message = requests.get(user_url).json()
    try:
        if ('uuid' in user_message) and ('message' in user_message):
            uuid = user_message.get('uuid')
            message = str(user_message.get('message'))
            bot_response = query_engine.query(message)
            requests.get(response_url.format(uuid, bot_response))

    except Exception as e:
        print(f"Invalid User Message : {user_message}")

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /chat/fetch-api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001997D034070>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))